# Processing plants data from the Energy Information Agency

## As always, import the required modules


In [1]:
import geopandas as gpd
import pandas as pd

Here we use geopandas to read in the downloaded power plants shapefile.

In [2]:
pplants = gpd.read_file(r'.\data\PowerPlants_US_EIA\PowerPlants_US_202001.shp')


Checking out just a bit of the structure, we can see that the source_des column has what we need. However...  It appears to have the different fuel sources in a comma separated string.

In [3]:
pplants.head(3)

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,StateName,Zip,Street_Add,...,Geo_MW,Bat_MW,Other_MW,source_des,tech_desc,Source,Period,Latitude,Longitude,geometry
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,Alabama,35476,19001 Lock 17 Road,...,None,None,None,Hydroelectric = 53 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,33.458665,-87.356823,POINT (-87.35682 33.45867)
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,Alabama,36512,North Highway 43,...,None,None,None,"Coal = 1118.5 MW, Natural Gas = 1268.4 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",202001,31.006900,-88.010300,POINT (-88.01030 31.00690)
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,Alabama,36092,750 Bouldin Dam Road,...,None,None,None,Hydroelectric = 224.1 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,32.583889,-86.283056,POINT (-86.28306 32.58389)


So we don't want want the ' MW' nor and spaces after the commas as this will mess up the formatting of the strings.
Let's nip that in the bud early on.

In [11]:
pplants["source_des_noMW"] = pplants['source_des'].replace({' MW': ''}, regex=True)
pplants["source_des_noMW"] = pplants['source_des_noMW'].replace({', ': ','}, regex=True)

This was an early attempt. It left us with a list and not the key:value pairs we are really looking for

In [13]:
pplants['fuel'] = pplants["source_des_noMW"].str.split(',',expand=False) 

Looking now, we can see that our string is now a list of strings.

In [14]:
pplants[['source_des_noMW','fuel']].head(3)

,source_des_noMW,fuel
0,Hydroelectric = 53,[Hydroelectric = 53]
1,"Coal = 1118.5,Natural Gas = 1268.4","[Coal = 1118.5, Natural Gas = 1268.4]"
2,Hydroelectric = 224.1,[Hydroelectric = 224.1]


We want the column with our fuel to be in a format with some key:value pairs where our fuel type is the key and the MW is our value.
So lets use the apply lamda function to apply a for loop to each string, first splitting the string, then looping over the results there to create a dictionary.


In [19]:
pplants['fuel_source'] = pplants['source_des_noMW'].apply(lambda x: dict(kv.split(' = ') for kv in x.split(',')))


In [20]:
pplants[['source_des_noMW','fuel_source']].head(3)

,source_des_noMW,fuel_source
0,Hydroelectric = 53,{'Hydroelectric': '53'}
1,"Coal = 1118.5,Natural Gas = 1268.4","{'Coal': '1118.5', 'Natural Gas': '1268.4'}"


Now we just want to rename our columns

In [21]:
pplants.rename({'Plant_Code': 'code', 'Plant_Name': 'plant_name','Total_MW': 'capacity_mw'}, axis=1, inplace=True)

In [22]:
pplants.head(2)

,code,plant_name,Utility_Na,Utility_ID,sector_nam,City,County,StateName,Zip,Street_Add,...,source_des,tech_desc,Source,Period,Latitude,Longitude,geometry,source_des_noMW,fuel,fuel_source
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,Alabama,35476,19001 Lock 17 Road,...,Hydroelectric = 53 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,33.458665,-87.356823,POINT (-87.35682 33.45867),Hydroelectric = 53,[Hydroelectric = 53],{'Hydroelectric': '53'}
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,Alabama,36512,North Highway 43,...,"Coal = 1118.5 MW, Natural Gas = 1268.4 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",202001,31.006900,-88.010300,POINT (-88.01030 31.00690),"Coal = 1118.5,Natural Gas = 1268.4","[Coal = 1118.5, Natural Gas = 1268.4]","{'Coal': '1118.5', 'Natural Gas': '1268.4'}"


We are also going to drop all the unnecessary columns.

In [23]:
pplants = pplants[['code','plant_name','capacity_mw','fuel_source','geometry']]

In [26]:
pplants.head(10)

,code,plant_name,capacity_mw,fuel_source,geometry
0,2,Bankhead Dam,53.0,{'Hydroelectric': '53'},POINT (-87.35682 33.45867)
1,3,Barry,2386.9,"{'Coal': '1118.5', 'Natural Gas': '1268.4'}",POINT (-88.01030 31.00690)
2,4,Walter Bouldin Dam,224.1,{'Hydroelectric': '224.1'},POINT (-86.28306 32.58389)
3,7,Gadsden,130.0,{'Natural Gas': '130'},POINT (-85.97080 34.01280)
4,9,Copper,63.0,{'Natural Gas': '63'},POINT (-106.37500 31.75690)
5,10,Greene County,1256.1,{'Natural Gas': '1256.1'},POINT (-87.78110 32.60170)
6,11,H Neely Henry Dam,71.1,{'Hydroelectric': '71.1'},POINT (-86.05240 33.78450)
7,12,Holt Dam,48.0,{'Hydroelectric': '48'},POINT (-87.44950 33.25530)
8,13,Jordan Dam,129.2,{'Hydroelectric': '129.2'},POINT (-86.25480 32.61890)
9,14,Logan Martin Dam,129.0,{'Hydroelectric': '129'},POINT (-86.33755 33.42588)


And finally, we are going to going to write out the file to our web app's "data" folder

In [25]:
pplants.to_file("../data/plants.geojson", driver='GeoJSON')